In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline 

In [2]:
df = pd.read_csv('data_train.csv')
df.head()

,Input_current,Output_voltage,Input_voltage,Duty_Cycle,Power,Irradiance,Temperature
0,6.818701,58.373037,29.782506,0.485,212.117783,1000,25
1,7.313437,53.815293,23.990136,0.557,180.751131,1000,60
2,6.056522,49.442379,24.298210,0.505,152.417456,836,60
3,5.735578,50.894729,26.911060,0.475,161.503052,836,48
4,5.376322,48.436413,26.198545,0.457,146.294526,755,48


Separamos as 4 entradas: Irradiance, Temperature, Tensão de Entrada e Corrente de entrada
Separamos a saida: DutyCycle

In [3]:
df_x = df.loc[:,['Irradiance','Temperature','Input_current','Input_voltage']] #Seleciona por nome
df_y = df.loc[:,['Duty_Cycle']] #Seleciona por nome
print(df_x[:5])
print(df_y[:5])
print(df_x.shape)

   Irradiance  Temperature  Input_current  Input_voltage
0        1000           25       6.818701      29.782506
1        1000           60       7.313437      23.990136
2         836           60       6.056522      24.298210
3         836           48       5.735578      26.911060
4         755           48       5.376322      26.198545
   Duty_Cycle
0       0.485
1       0.557
2       0.505
3       0.475
4       0.457
(2093, 4)


In [55]:
x=np.asarray(df_x)
y=np.asarray(df_y)
print(x.shape,y.shape)

(2093, 4) (2093, 1)


Após a obtenção dos dados do arquivo, é feito a separação do conjunto de dados entre treinamento e teste.

In [56]:
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.5,random_state=1)

Em seguida, cria-se um modelo para a normalização dos dados para que se obtenha os dados com media zero e variancia unitaria. Assim  são normalizados os dados de entrada de treinamento e de teste.

In [57]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [58]:
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(y_train.shape)

(1046, 1)


Em seguida são levantados os parametros desejados da rede neural e feito o seu treinamento

In [74]:
modelo = MLPRegressor(activation='tanh', solver='lbfgs', hidden_layer_sizes=(4,4))
modelo.fit(X_train,y_train)
print(modelo.score(X_test, y_test))

print(modelo.n_layers_)
print(modelo.coefs_)

0.9980261740720318
4
[array([[-0.25697724,  0.40848299,  0.12275243, -0.44942224],
       [ 0.13079062, -0.35586602, -0.07893398,  0.01953215],
       [ 0.5889776 , -0.90962146,  0.39079071, -0.58884994],
       [ 0.63646316, -0.7396986 , -0.18910467,  0.19588317]]), array([[ 0.55415607,  0.15630137, -0.0551677 ,  0.40077632],
       [-0.12060126,  0.18979044, -0.1763701 , -0.69339697],
       [ 0.0080549 , -0.75949958,  0.17613204,  0.47601213],
       [-0.16060713, -0.19937926,  0.35175575,  0.06054658]]), array([[-0.170398  ],
       [-0.43869375],
       [-0.33270128],
       [ 0.2332325 ]])]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [75]:
modelo.out_activation_

'identity'

Geração de arquivos .csv com os valores de pesos e bias do modelo criado para import no matlab

In [76]:
modelo.coefs_

[array([[-0.25697724,  0.40848299,  0.12275243, -0.44942224],
        [ 0.13079062, -0.35586602, -0.07893398,  0.01953215],
        [ 0.5889776 , -0.90962146,  0.39079071, -0.58884994],
        [ 0.63646316, -0.7396986 , -0.18910467,  0.19588317]]),
 array([[ 0.55415607,  0.15630137, -0.0551677 ,  0.40077632],
        [-0.12060126,  0.18979044, -0.1763701 , -0.69339697],
        [ 0.0080549 , -0.75949958,  0.17613204,  0.47601213],
        [-0.16060713, -0.19937926,  0.35175575,  0.06054658]]),
 array([[-0.170398  ],
        [-0.43869375],
        [-0.33270128],
        [ 0.2332325 ]])]

In [77]:
coef_0=np.asarray(modelo.coefs_[0])
coef_1=np.asarray(modelo.coefs_[1])
coef_2=np.asarray(modelo.coefs_[2])
coefs=np.concatenate((coef_0,coef_1),axis=1)
coefs=np.concatenate((coefs,coef_2),axis=1)
np.savetxt("pesos.csv", coefs, delimiter=",")

bias_0=np.asarray(modelo.intercepts_[0])
bias_1=np.asarray(modelo.intercepts_[1])
bias_2=np.asarray(modelo.intercepts_[2])
bias=np.concatenate((bias_0,bias_1))
bias=np.concatenate((bias,bias_2))
np.savetxt("bias.csv", bias, delimiter=",")
print(modelo.intercepts_)

[array([0.66272963, 0.2388292 , 0.73559444, 0.64678621]), array([ 0.32266736, -0.36475369,  0.1124503 ,  0.69468394]), array([0.14205554])]


In [24]:
y_hat = modelo.predict(X_test)
y_hat[0]

0.3713631994579457

In [25]:
y_test[0]

array([0.375])

In [26]:
np.mean((y_hat-y_test)**2)

0.021129441078772303

In [27]:
valid = pd.read_csv('data.csv')
valid.head()

,Input_current,Output_voltage,Input_voltage,Duty_Cycle,Power,Irradiance,Temperature
0,6.818671,58.373244,29.782580,0.485,212.118655,1000.0,25
1,7.206004,57.995148,28.079099,0.513,210.170608,1000.0,27
2,7.651242,59.317953,27.550531,0.537,220.197268,1048.0,27
3,7.030870,60.620215,31.017551,0.483,228.780529,1048.0,17
4,4.054196,45.744741,30.788208,0.325,130.447698,588.0,17


In [28]:
valid_x = valid.loc[:,['Irradiance','Temperature','Input_current','Input_voltage']] #Seleciona por nome
valid_y = valid.loc[:,['Duty_Cycle']] #Seleciona por nome
print(valid_x[:5])
print(valid_y[:-1])
print(valid_x.shape)

   Irradiance  Temperature  Input_current  Input_voltage
0      1000.0           25       6.818671      29.782580
1      1000.0           27       7.206004      28.079099
2      1048.0           27       7.651242      27.550531
3      1048.0           17       7.030870      31.017551
4       588.0           17       4.054196      30.788208
      Duty_Cycle
0          0.485
1          0.513
2          0.537
3          0.483
4          0.325
5          0.361
6          0.529
7          0.501
8          0.117
9          0.179
10         0.417
11         0.403
12         0.503
13         0.465
14         0.507
15         0.431
16         0.385
17         0.377
18         0.411
19         0.487
20         0.505
21         0.475
22         0.487
23         0.517
24         0.389
25         0.315
26         0.419
27         0.437
28         0.517
29         0.469
...          ...
1969       0.000
1970       0.000
1971       0.000
1972       0.000
1973       0.000
1974       0.000
1975       0

In [29]:
x_val=np.asarray(valid_x)
y_val=np.asarray(valid_y)
print(x_val.shape,y_val.shape)

(2000, 4) (2000, 1)


In [32]:
X_val = scaler.transform(x_val)
modelo.score(X_val,y_val)

-4.848383128539281

In [34]:
Y_val=modelo.predict(X_val)
print(Y_val)

[0.48847427 0.51385134 0.53570434 ... 0.72852261 0.72852261 0.72852261]


In [429]:
y_val


array([[0.485],
       [0.513],
       [0.537],
       ...,
       [0.   ],
       [0.   ],
       [0.   ]])

In [430]:
np.mean((Y_val-y_val)**2)

0.1315794442522724

In [35]:
print('Accuracy testing : {:.3f}'.format(modelo.score(X_val, y_val)))

Accuracy testing : -4.848
